In [1]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=a96a9726d2d07644ab4bd8c87fbbd0efed77e988d9d487c53ccb8955fd5ec8df
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
from google.colab import files
src = list(files.upload().values())[0]
#open('/content/dataloader.ipynb','r').read(src)
#import PolyDatasetShape, PolyDatasetStitch, PolyDatasetRefine

with open('/content/dataloader.ipynb', "r") as file1:
  dataloader = file1.read()
  #from dataloader import PolyDatasetShape, PolyDatasetStitch, PolyDatasetRefine

with open('/content/utils.ipynb', "r") as file2:
  utils = file2.read()
  #from utils import ReplayBuffer, weights_init_normal, LambdaLR

with open('/content/models.ipynb', "r") as file3:
  models = file3.read()
  #from models import GeneratorCoarse, Discriminator

Saving dataloader.py to dataloader.py
Saving dataloader.ipynb to dataloader.ipynb
Saving models.py to models.py
Saving utils.py to utils.py
Saving utils.ipynb to utils.ipynb
Saving models.ipynb to models.ipynb


In [3]:
dataloader = open('dataloader.ipynb', 'wb').write(src)
models = open('models.ipynb', 'wb').write(src)
utils = open('utils.ipynb', 'wb').write(src)

In [4]:
c
from dataloader import PolyDatasetShape, PolyDatasetStitch, PolyDatasetRefine
from models import GeneratorCoarse, Discriminator
from utils import ReplayBuffer, weights_init_normal, LambdaLR

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
def get_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--j', type = int, default = 0) # number of workers used for loading data
    parser.add_argument('--b', type = int, default = 1) # batch size
    parser.add_argument("--dataroot")#, default = "Data")
    parser.add_argument("--datamode", default = "Train")
    parser.add_argument("--stage", default = "Refine", help = 'Shape, Stitch, Refine')
    parser.add_argument("--data_list", default = "train_pairs.txt")
    parser.add_argument("--radius", type = int, default = 5) # skeleton width
    parser.add_argument("--grid_size", type = int, default = 5)
    parser.add_argument('--lr', type = float, default = 0.0002, help = 'Initial Learning Rate')
    parser.add_argument("--display_count", type = int, default = 1000)
    parser.add_argument("--save_count", type = int, default = 100)
    parser.add_argument("--shuffle", action ='store_true', help = 'Shuffle Input Data')
    parser.add_argument("--epochs", type = int, default = 45)
    parser.add_argument("--input_channel", type = int, default = 6)
    parser.add_argument("--decay_epoch", type = int, default = 10)
    parser.add_argument('--results', type = str, default = 'Results', help = 'Save Results')
    parser.add_argument("--critic", type = int, default = 10) # Number of times after which to update Discriminator.
    parser.add_argument("--save_model", type = int, default = 2)
    parser.add_argument('-f')
    opt = parser.parse_args()
    return opt

In [6]:
def train(opt, train_loader, netG, netD):
    epoch = 0
    n_epochs = opt.epochs
    decay_epoch = opt.decay_epoch
    batchSize = opt.b
    size = 128
    input_nc = opt.input_channel
    output_nc = 3
    lr = opt.lr
    if opt.stage != "Refine":
        nRow = 3
    else:
        nRow = 4
    
    criterion_GAN = torch.nn.MSELoss()
    criterion_identity = torch.nn.L1Loss()

    optimizer_G = torch.optim.Adam(netG.parameters(),lr = lr, betas = (0.5, 0.999))
    optimizer_D = torch.optim.Adam(netD.parameters(), lr = lr, betas = (0.5, 0.999))

    lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda = LambdaLR(n_epochs, epoch, decay_epoch).step)
    lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda = LambdaLR(n_epochs, epoch, decay_epoch).step)

    # Inputs & targets memory allocation
    Tensor = torch.cuda.FloatTensor
    input_A = Tensor(batchSize, input_nc, size, size)
    target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad = False)
    target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad = False)

    fake_buffer = ReplayBuffer()
    
    for epoch in range(0, n_epochs):
        gc.collect()
        Source = iter(train_loader)
        avg_loss_g = 0
        avg_loss_d = 0
        for i in range(0, len(train_loader)):
            netG.train()
            target_real = Variable(torch.ones(1, 1), requires_grad = False).cuda()
            target_fake = Variable(torch.zeros(1, 1), requires_grad = False).cuda()
            optimizer_G.zero_grad()

            if opt.stage != "Refine":
                src, mask, style_img, target, gt_cloth, skel, cloth = Source.next()
                src, mask, style_img, target, gt_cloth, skel, cloth = Variable(src.cuda()), Variable(mask.cuda()), Variable(style_img.cuda()), 
                Variable(target.cuda()), Variable(gt_cloth.cuda()), Variable(skel.cuda()), Variable(cloth.cuda())
            else:
                src, mask, style_img, target, gt_cloth, wrap, diff, cloth = Source.next()
                src, mask, style_img, target, gt_cloth, wrap, diff, cloth = Variable(src.cuda()), Variable(mask.cuda()), Variable(style_img.cuda()), 
                Variable(target.cuda()), Variable(gt_cloth.cuda()), Variable(wrap.cuda()), Variable(diff.cuda()), Variable(cloth.cuda())

            # Inverse identity
            if opt.stage == "Shape":
                gen_targ,_,_,_,_,_,_ = netG(skel, cloth) # src,conditions
            elif opt.stage == "Stitch":
                gen_targ,_,_,_,_,_,_ = netG(src, style_img, skel)
            elif opt.stage == "Refine":
                gen_targ,_,_,_,_,_,_ = netG(diff, wrap)
                
            pred_fake = netD(gen_targ)
            
            if opt.stage == "Shape":
                loss_GAN = 10 * criterion_GAN(pred_fake, target_real) + 10 * criterion_identity(gen_targ, gt_cloth)
            elif opt.stage == "Stitch" or opt.stage == "Refine":
                loss_GAN = 10 * criterion_GAN(pred_fake, target_real) + 10 * criterion_identity(gen_targ, target)

            loss_G = loss_GAN
            loss_G.backward()

            optimizer_G.step()        
            #############################################
            optimizer_D.zero_grad()

            if opt.stage == "Shape":
                pred_real = netD(gt_cloth)
            elif opt.stage == "Stitch" or opt.stage == "Refine":
                pred_real = netD(target)
            
            loss_D_real = criterion_GAN(pred_real, target_real)

            # Fake loss
            gen_targ = fake_buffer.push_and_pop(gen_targ)
            pred_fake = netD(gen_targ.detach())
            loss_D_fake = criterion_GAN(pred_fake, target_fake)

            # Total loss
            loss_D = (loss_D_real + loss_D_fake) * 0.5
            loss_D.backward()
            if (i + 1) % opt.critic == 0:
                optimizer_D.step()

            avg_loss_g = (avg_loss_g + loss_G) / (i+1) 
            avg_loss_d = (avg_loss_d + loss_D) / (i+1) 

            if (i + 1) % 100 == 0:
                print("Epoch: (%3d) (%5d/%5d) Loss: (%0.0003f) (%0.0003f)" % (epoch, i + 1, len(train_loader), avg_loss_g * 1000, avg_loss_d * 1000))


            if (i + 1) % opt.display_count == 0:  
                if opt.stage == "Shape":
                    pic = (torch.cat([style_img, gen_targ, cloth, skel, target, gt_cloth], dim = 0).data + 1) / 2
                elif opt.stage == "Stitch":
                    pic = (torch.cat([src, gen_targ, cloth, skel, target, gt_cloth], dim = 0).data + 1) / 2
                elif opt.stage == "Refine":
                    pic = (torch.cat([wrap, diff, gen_targ, target], dim = 0).data + 1) / 2
                save_dir = "{}/{}".format(os.getcwd(), opt.results)
                os.mkdir(save_dir)
                save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, i + 1, len(train_loader)), nrow = nRow)
        if (epoch + 1) % opt.save_model == 0:
            save_dir = "{}/{}".format(os.getcwd(), opt.results)
            torch.save(netG.state_dict(), '{}/Gan_{}.pth'.format(save_dir, epoch))
        # Update learning rates
        lr_scheduler_G.step()
        lr_scheduler_D.step()

In [8]:
def main():
    opt = get_opt()
    print(opt)
    print("Start to train stage: %s" % (opt.stage))
        
    # create dataset 
    if opt.stage == "Shape":
      dataset = PolyDatasetShape(128)
      train_loader = DataLoader(dataset, batch_size = opt.b, shuffle = False, num_workers = opt.j, drop_last = True, pin_memory = True)
                
    elif opt.stage == "Stitch":
        dataset = PolyDatasetStitch(128)
        train_loader = DataLoader(dataset, batch_size = opt.b, shuffle = False, num_workers = opt.j, drop_last = True, pin_memory = True)

    elif opt.stage == "Refine":
      dataset = PolyDatasetRefine(128)
      train_loader = DataLoader(dataset, batch_size = opt.b, shuffle = False, num_workers = opt.j, drop_last = True, pin_memory = True)
   
    else:
      sys.exit("Please mention the Stage from [Shape, Stitch, Refine]")
        
    if not os.path.exists(opt.results):
        os.makedirs(opt.results)
    netG = GeneratorCoarse(opt.input_channel, 3)
    netD = Discriminator()   
    # create model & train & save the final checkpoint
    netG.cuda()
    netD.cuda()
    netG.apply(weights_init_normal)
    netD.apply(weights_init_normal)    
    train(opt, train_loader, netG, netD)
    print('Finished training %s!' % (opt.stage))

if __name__ == "__main__":
    main()

Namespace(b=1, critic=10, data_list='train_pairs.txt', datamode='Train', dataroot=None, decay_epoch=10, display_count=1000, epochs=45, f='/root/.local/share/jupyter/runtime/kernel-0f1ab72e-3751-43b1-a63d-bdf1582e664a.json', grid_size=5, input_channel=6, j=0, lr=0.0002, radius=5, results='Results', save_count=100, save_model=2, shuffle=False, stage='Refine')
Start to train stage: Refine
[] []


/content/utils.py:51: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Finished training Refine!
